# Powerpoint Slides Summarizer

This converts a Power Point presentation into notes that a student can easily skim through.

Concepts Used:
- Converting Contents of PPT to text via python-pptx
- User and System Prompts
- Use of Open AI GPT-4o-mini via API key


In [3]:
!pip install python-pptx

In [4]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [6]:
# creating an instance
openai = OpenAI()

In [9]:
from pptx import Presentation

class PowerPoint():
    def __init__(self,ppt):
        """
        Creates a PowerPoint object, with name and text.
        """
        self.ppt = ppt
        self.title = os.path.basename(ppt)
        self.text = self.extract_text()

    def extract_text(self):
        """
        Extracts text from powerpoint.
        """
        prs = Presentation(self.ppt)
        text_content = []
    
        for slide in prs.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text_content.append(shape.text)
    
        return "\n".join(text_content)
    

In [8]:
system_prompt = "You are an assistant that analyzes the contents \
of a PowerPoint presentation, and provides a summary in the style of \
a cheat-sheet, for students to easily learn key concepts from.\
You are to ignore text that might be navigation-related\
and respond in Markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of PowerPoints:

def user_prompt_for(powerpoint):
    user_prompt = f"You are looking at a website titled {powerpoint.title}"
    user_prompt += "\nThe contents of this powerpoint are as follows; \
please provide a summary of the content in markdown. \
If it includes a question bank, add that along with short answers too.\n\n"
    user_prompt += powerpoint.text
    return user_prompt

In [14]:
def messages_for(powerpoint):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(powerpoint)}
    ]

In [15]:
# And now: call the OpenAI API. 

def summarize(powerpoint_path):
    powerpoint = PowerPoint(powerpoint_path)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(powerpoint)
    )
    return response.choices[0].message.content

In [18]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
ppt_file = "Theoretical Perspectives on Media and Technology.pptx" 
display_summary(ppt_file)

# Cheat-Sheet: Theoretical Perspectives on Media and Technology

## Introduction to Media and Technology
- **Impact of Technology:** Transforms daily interactions; enhances connectivity through social media (e.g., Facebook).
- **Influence:** Affects communication, socialization, and perception of the world.

## Technological Inequality
- **Digital Divide:** Gap between those with and without access to technology.
- **Knowledge Gap:** Differences in knowledge based on socioeconomic status.

## Evolution of Media
- **Interwoven Relationship:** Media and technology influence each other.
- **Historical Impact:** The printing press revolutionized information; rapid technological advances continue to change media.

## Types of Media Technology
- **Examples:** Comics, Movies, Websites, Newspapers.
- **Concerns:** Link between violent video games and aggressive behavior (Anderson and Bushman, 2001).
- **Regulations:**
  - ERSB rating system for video games (1994).
  - Supreme Court ruling on minors accessing video games (2011).

## Global Implications of Media and Technology
- **Connecting People:** Thomas Friedman’s "The World is Flat" emphasizes the role of computing and the internet in globalization.

## Case Studies
- **Government Censorship:** Local alternatives to Western social media; using social media for critique amidst restrictions.
- **Mobile Phones in Sub-Saharan Africa:** Increased market access and money transfers.
- **China and the Internet:** Examples of information control and technology's role in society.

## Theoretical Perspectives

### Functionalism
- **Media Functions:**
  - **Commercial:** Advertising platform.
  - **Entertainment:** Various forms of media.
  - **Social-Norm:** Socializing tool.
  - **Life-Changing:** Enhances access to information.
- **Manifest Functions:** Shape reality perceptions, impact language, and social cohesion.
- **Dysfunctions:** Over-commercialization, excessive media consumption, digital addiction, privacy concerns.

### Conflict Theory
- **Gatekeeping:** Influential figures control media; selective information is given to the public.
- **Digital Control:** Surveillance and monitoring of behavior.
- **Unbalanced Political Arena:** Financial power influences media exposure.

### Feminist Perspective
- **Media Representation:** Exclusion of diverse beauty and reinforcement of stereotypes.
- **Cyber-Feminism:** Advocacy for women's representation and rights in the digital realm.
- **Movements:** #MeToo, body positivity, blogs by Iraqi women.

### Symbolic Interactionism
- **Symbolism of Technology:** Represents wealth/status differently perceived by various groups (Neo-Luddites vs. Technophiles).
- **Media's Role:** Shapes how reality is understood and creates societal frameworks for comparison.

## Questions and Answers
1. **What is Technological Stratification?**
   - Differences in access to technology leading to unequal opportunities.

2. **What is the 'Digital Divide'?**
   - The gap between those who have access to digital technology and those who do not.

3. **What are the risks associated with violent video games?**
   - Linked to increased aggression and desensitization to violence.

4. **What does 'gatekeeping' refer to?**
   - The process of filtering and determining what information is presented in the media.

5. **What is the significance of the ERSB rating system?**
   - It provides a framework for assessing the content of video games to inform consumers.

6. **Define Cyber-Feminism.**
   - Promoting feminism and women's representation within the digital and online spaces.

7. **What does 'panoptic surveillance' refer to?**
   - Constant digital monitoring without direct communication, such as through security cameras.

---

This cheat-sheet provides an overview of the critical themes explored in the theoretical perspectives on media and technology. Be sure to understand these concepts for a holistic grasp of media studies!